In [17]:
from __future__ import annotations
from typing import Iterable, Literal
from typing_extensions import Self


def parser(file = 'input.txt'):
    with open(file, 'r') as file:
        # for line in file:
        #     yield line.strip('\n')
        workflows_raw, cases = file.read().split('\n\n')
        workflows = {}
        for wf in workflows_raw.split('\n'):
            key = wf[:wf.index('{')]
            rules_raw = wf[wf.index('{')+1:-1]
            rules = [(rule[0], rule[1], int(rule[2:rule.index(':')]), rule[rule.index(':')+1:]) if ':' in rule else rule for rule in rules_raw.split(',')]
            possible_destinations = [r[-1] for r in rules[:-1]] + [rules[-1]]
            # print(key, rules, possible_destinations)
            if key in workflows:
                raise Exception()
            workflows[key] = (rules, possible_destinations)
        cases = [{combo[0]:int(combo[2:]) for combo in case[1:-1].split(',')} for case in cases.split('\n')]
        # print(cases)
        return workflows, cases


def solver(workflows, cases):
    # print([key for key, (rules, dests) in workflows.items() if 'A' in dests])
    summed = 0
    for case in cases:
        # print("\nCASE")
        curr = 'in'
        while not curr or curr not in 'AR':
            # print(curr)
            rules, _ = workflows[curr]
            else_dest = rules[-1]
            
            for rule in rules[:-1]:
                var, op, value, new = rule
                if {'>': int.__gt__, '<': int.__lt__}[op](case[var], value):
                    curr = new
                    break
            else:
                curr = else_dest
        if curr == 'A':
            # print(case, curr)
            summed += sum(case.values())
    return summed


if __name__ == '__main__':
    test_p1 = solver(*parser('test_input.txt'))
    print(test_p1)
    assert test_p1 == 19114, test_p1
    print(solver(*parser()))

    # test_p2 = solver(parser('test_input.txt'))
    # test_p2 = solver(parser('test_input_p2.txt'))
    # print(test_p2)
    # assert test_p2 == None, test_p2
    # print(solver(parser()))

19114
472630


In [5]:
N = int | Literal['inf']

class infrange:
    start: N
    end: N

    def __init__(self, start: N, end: N):
        self.start = start
        self.end = end
    
    def __and__(self, other: Self):
        match (self.start, other.start):
            case 'inf', 'inf':
                start = 'inf'
            case _, 'inf':
                start = self.start
            case 'inf', _:
                start = other.start
            case a, b:
                start = max(a, b)
        
        match (self.end, other.end):
            case 'inf', 'inf':
                end = 'inf'
            case _, 'inf':
                end = self.end
            case 'inf', _:
                end = other.end
            case a, b:
                end = min(a, b)
        
        return infrange(start, end)
    
    def __repr__(self):
        return f'infrange(start={self.start}, end={self.end})'

print(infrange('inf', 5) & infrange(3, 'inf'))

infrange(start=3, end=5)
